In [104]:
# import sys
from pyspark.sql import SparkSession

In [105]:
spark = SparkSession.builder \
    .appName("Spark Session") \
    .enableHiveSupport() \
    .getOrCreate()

In [106]:
df = spark.sql("REFRESH TABLE silver.nubank_extrato")
df = spark.sql("REFRESH TABLE silver.nubank_fatura")

In [107]:
df = spark.sql("""
SELECT
Data,
Data_Pgto,
DebitoCredito,
Entidade,
Tipo,
Valor
FROM silver.nubank_extrato        
WHERE Identificador NOT IN
(
    SELECT e.Identificador FROM silver.nubank_extrato e
    INNER JOIN 
          (
          SELECT DISTINCT Data_Pgto FROM silver.nubank_fatura 
          WHERE Descricao != 'Pagamento recebido'
          ) f ON SUBSTRING(f.Data_Pgto,0,7) = SUBSTRING(e.Data,0,7)
    WHERE e.Descricao = 'Pagamento de fatura'
)
UNION ALL
SELECT 
Data,
Data_Pgto,
DebitoCredito,
Entidade,
Tipo,
Valor
FROM silver.nubank_fatura
Where Descricao != 'Pagamento recebido'
""")


In [108]:
df.coalesce(1).write.mode("overwrite").format("parquet").saveAsTable(f"gold.nubank_movimentacao")

In [109]:
spark.sql("select sum(valor) from gold.nubank_movimentacao").show()

+-----------------+
|       sum(valor)|
+-----------------+
|7019.269999999879|
+-----------------+

